# TODO
- Load a prebuilt dataset.
- Build a neural network machine learning model that classifies images.
- Train this neural network.
- Evaluate the accuracy of the model.

## Key Points:
- `Conv2D` layers extract spatial features.
- `MaxPooling2D` reduces the spatial dimensions to focus on the most important features.
- `Flatten` converts 2D feature maps into 1D for fully connected layers.
- `Dense` layers combine features and make predictions.
- `Dropout` helps prevent overfitting.
- `BatchNormalization` stabilizes training.

In [2]:
import tensorflow as tf

In [3]:
print("Tensorflow version: ",tf.__version__)

Tensorflow version:  2.17.0


## Load a dataset

The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by 255.0. This also converts the sample data from integers to floating-point numbers:


In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


## Build a machine learning model
- Sequential API: Suitable for simple, linear stacks of layers.
- Functional API: More flexible, allowing for complex architectures like multi-input/output models, residual connections, etc.

Sequential is useful for stacking layers where each layer has one input tensor and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the `Flatten`, `Dense`, and `Dropout` layers.

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(18)
    ])

In [9]:
predictions = model(x_train[:1]).numpy()

In [10]:
predictions

array([[-0.06544951,  0.01938036, -0.9274679 ,  0.24186485, -0.06771566,
        -0.10151345, -0.13855836, -0.4946216 , -0.4691984 , -0.32899454,
         0.48661968,  0.6311363 , -0.3962165 , -0.15115657,  0.24754627,
         0.81437457, -0.5447697 ,  0.38291305]], dtype=float32)

### The `tf.nn.softmax` function converts these logits to probabilities for each class:

In [12]:
tf.nn.softmax(predictions).numpy()

array([[0.0495233 , 0.05390768, 0.02091413, 0.06734022, 0.0494112 ,
        0.04776912, 0.04603189, 0.03224204, 0.03307224, 0.0380499 ,
        0.0860142 , 0.09938776, 0.03557618, 0.04545561, 0.06772389,
        0.11937471, 0.03066504, 0.07754093]], dtype=float32)

### Define a loss function for training using losses.SparseCategoricalCrossentropy:
- Loss: categorical_crossentropy for multi-class classification
- sparse_categorical_crossentropy if labels are integers.

In [16]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [17]:
loss_fn(y_train[:1], predictions).numpy()

3.041376

The loss is zero if the model is sure of the correct class.

### Before you start training:
- configure model using keras `model.compile`
- set the optimizer class to `adam` - Optimizer: Adam, SGD, RMSprop (Adam is commonly used).
- set the loss to loss_fn
- specify a metric to be evaluated for the model by setting the `metrics=['accuracy']`

In [19]:
model.compile(optimizer='adam',
             loss=loss_fn,
              metrics=['accuracy']
             )

## Train and evaluate your model
 - `Model.fit` method to adjust your model parameters and minimize the loss:
 - Using model.fit() to train the model.
- Specifying batch_size, epochs, and using validation data.
- Callbacks like EarlyStopping and ModelCheckpoint to prevent overfitting and save the best model.
### Evaluation: Using model.evaluate() to assess the model’s performance on test data.

In [20]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8506 - loss: 0.5270
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9535 - loss: 0.1572
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9667 - loss: 0.1098
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9720 - loss: 0.0892
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9772 - loss: 0.0737


<b> The Model.evaluate method checks the model's performance, usually on a validation set or test set.</b>

In [21]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - 2ms/step - accuracy: 0.9796 - loss: 0.0703


[0.07026389986276627, 0.9796000123023987]

<b>The image classifier is now trained to ~98% accuracy on this dataset.</b>

<b>If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:</b>

In [22]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [23]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 18), dtype=float32, numpy=
array([[4.00776790e-09, 5.18754550e-09, 1.68514219e-07, 3.95309500e-04,
        3.46160839e-12, 3.32354546e-07, 1.93528502e-13, 9.99601543e-01,
        7.24052654e-07, 1.94429003e-06, 1.01841348e-11, 1.50917074e-11,
        1.97901981e-12, 2.27695163e-12, 4.34487776e-12, 5.58653956e-11,
        2.55098061e-11, 1.26900825e-11],
       [2.91046547e-08, 2.91839547e-06, 9.99982476e-01, 1.33538269e-05,
        2.42419427e-16, 4.05295131e-09, 5.57187940e-09, 1.09472429e-13,
        1.15936484e-06, 3.96687078e-14, 8.82293023e-16, 4.05650613e-18,
        9.66475842e-18, 1.74350483e-17, 1.19634644e-17, 3.78659896e-15,
        1.56838076e-16, 4.35369403e-16],
       [2.54886663e-06, 9.99261081e-01, 1.59540810e-04, 4.65186122e-06,
        2.24279611e-05, 4.73507771e-06, 8.47010506e-06, 4.54083289e-04,
        8.14542436e-05, 7.31780858e-07, 1.31829738e-08, 3.89068830e-08,
        1.45355550e-08, 2.40583695e-08, 2.24958541e-08, 1.32071856e-08,
     